# OCR作業用notebook
ここで試したものをスクリプト化したものが`post_cloudvision.py`   （になる予定）  
このときはまだデータセットの構成が定まっていなかった


# TODO

- notebookからスクリプトに直す
    - 開始・終端ページの指定が必要
    - １巻ごとのスクリプトにすべき

In [15]:
import base64
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import os
import sys
import re
import codecs
import unicodedata
import nltk
import cv2
%matplotlib inline

url = 'https://vision.googleapis.com/v1/images:annotate?key='
api_key = os.env['cloud_vision_api_key']

# コマ確認

In [ ]:
file_path = './output/newgame4/0009/3.jpg'
image = Image.open(file_path, 'r').convert('RGB')
plt.imshow(image)

# CloudVision用公式スクリプトを利用
`cloud-vision/python/utils/generatejson.py` を使えば  
input用ファイルでファイルパスとクエリ指定するだけでjson化してくれるらしい

In [26]:
# 課金発生ポイント
raise Exception

# テスト実行
data = open('/Users/ichinari_sato/workspace/git/cloud-vision/python/utils/vision.json', 'rb').read()
response = requests.post(url='https://vision.googleapis.com/v1/images:annotate?key='+api_key,
                         data=data,
                         headers={'Content-Type': 'application/json'})
print response.text

{
  "responses": [
    {
      "textAnnotations": [
        {
          "locale": "ja",
          "description": "し▼▼▼進彗\nし…進捗\nどうですか!!\nM\n",
          "boundingPoly": {
            "vertices": [
              {
                "x": 253,
                "y": 39
              },
              {
                "x": 425,
                "y": 39
              },
              {
                "x": 425,
                "y": 217
              },
              {
                "x": 253,
                "y": 217
              }
            ]
          }
        },
        {
          "description": "し",
          "boundingPoly": {
            "vertices": [
              {
                "x": 404,
                "y": 39
              },
              {
                "x": 421,
                "y": 40
              },
              {
                "x": 420,
                "y": 66
              },
              {
                "x": 403,
                "y": 65
              }
        

In [33]:
responses = json.loads(response.text)

In [52]:
for i, r in enumerate(responses['responses']):
    print i, r['textAnnotations'][0]['description'].replace('\n','')


0 し▼▼▼進彗し…進捗どうですか!!M
1 ん/avayキャラリーダーの仕事はまずADである私のグラフィックへの意図の理解その上でキャラモデルのチェックと管理うんうん
2 あとは各班との連携のための会議かなまあそこは私もいるし気楽にねう…うん
3 あ明日バ切だと思うけど大丈夫…そう?は丶大丈夫ですよ
4 Cははわかった…がんばって…ね!ままぁ…
5 はいあ青葉…ちゃん!よ丶進捗:
6 えっと…ごめんなさいと...キャラデザに手間取ったせいで/3D作業の方がそのまま遅れてて…...11日ほど追加お願いできません…か?
7 わかった…じゃあがんばってね…!し
8 ごめんなさい!一日追加でいいですに向Cへ!?


# inputファイルを自動生成する

ファイルパスはほぼ自動で決まっているものの，コマ数とかは結構ページによりまちまちなので，  
ファイルを辿って自動で読み込む必要がある  
あと全部ぶちこむと流石に不安．1巻1000コマ近くあるし．  
あとあとのOCRのことも考えたらページごとに行って，コマごとの出力ファイルを作るのがよさそう？  
と思ったら1ページを画像として投げたら4コマ分縦に台詞が繋がって出力されてしまった  
画像はコマ単位で投げるしか無い  
inputファイルはページ単位で作って，120ページ分をそれぞれリクエストすれば良さそう  
1リクエストの出力をダンプしておいて，ページ単位やコマ単位の判断はあとですればいいや  

In [19]:
base_path = '/Users/ichinari_sato/workspace/notebook/output/'
last_page = [0, 117, 118, 117, 117, 118]

for comic_num in range(1, 5+1):
    comic_path = os.path.join(base_path, 'newgame{}/'.format(comic_num))
    for page_file in os.listdir(comic_path):
        if not page_file.isdigit():
            continue
        page = int(page_file)
        if page < 9 or last_page[comic_num] < page:
            continue
        page_path = os.path.join(comic_path, page_file)
        jpg_list = []
        for frame_file in os.listdir(page_path):
            if not frame_file.split(os.extsep)[0].isdigit():
                continue
            jpg_list.append(os.path.join(page_path, frame_file))
        input_path = '/Users/ichinari_sato/workspace/notebook/output/ocr_newgame{}/input_{}.txt'.format(comic_num, page)
        with open(input_path, 'w') as f:
            f.write('\n'.join(map(lambda x: x+' 5:1', jpg_list)))

開始ページはどの巻も一緒で9ページから，ただし最終ページは違うのでlast_pageで指定  
`ocr_newgame*`で以下を実行する

```
% for i in $(seq 9 $(expr `ls -1 *.txt |wc -l` + 8)); do python /Users/ichinari_sato/workspace/git/cloud-vision/python/utils/generatejson.py -i input_${i}.txt -o request_${i}.json ; done
```

ここまでで全jsonが完成, cloudVisionに投げるだけ  
API制限のため一気に全巻とかはやめておこう  

# Cloud Vision に投稿

In [31]:
# 課金発生ポイント
raise Exception # うっかり実行しないようにセーフティ

comic_num = 5
base_path = '/Users/ichinari_sato/workspace/notebook/output/'
ocr_path = os.path.join(base_path, 'ocr_newgame{}'.format(comic_num))
response_path = os.path.join(base_path, 'response_newgame{}'.format(comic_num))
for json_file in os.listdir(ocr_path):
    if not json_file.endswith('.json'):
        continue
    json_path = os.path.join(ocr_path, json_file)
    data = open(json_path, 'rb').read()
    response = requests.post(url='https://vision.googleapis.com/v1/images:annotate?key='+api_key,
                                                data=data,
                                                headers={'Content-Type': 'application/json'})
    frame_num = re.split(r'\.|_', json_file)[1]
    json_path = os.path.join(response_path, 'response{}.json'.format(frame_num))
    with codecs.open(json_path, 'w', 'utf-8') as f:
        f.write(response.text)

# 台詞にラベルを付ける

どの台詞がどのキャラか判定しておかないといけない  
こればっかりはやっぱり手でやらないとダメそう  
仕方ないので巻ごとにCSVで吐き出して，漫画を確認しながらExcel上で数値でラベル付けていこう  

**シリーズ名/line/巻数.csv**   
**ページ数, コマ数, 台詞, ラベル欄(吐き出し時は空白)**  

でラベル付け後にラベル欄が空白のままだったら（ゴミなので使えないから）削除するということで

| キャラ名    |ラベル |
|:---------- |  ----- |
|涼風青葉    | 0      |
|滝本ひふみ  | 1      |
|飯島ゆん     |2      |
|篠田はじめ  | 3      |
|八神コウ     |4      |
|遠山りん     |5      |
|阿波根うみこ |6      |
|葉月しずく   |7      |
|大和・クリスティーナ・和子 |8      |
|桜ねね       |9      |
|星川 ほたる | 10     |
|日高 ちなつ | 11     |
|望月 紅葉|12
|鳴海 ツバメ|13
|その他       |99     |

ラベル順はwikipediaの登場人物順。  
登場順でも良かったが、最初にその他に分類した人が重要人物になって後から追加する可能性もあったので（クリスとか）。  
結局どこかの段階では無理がくるので一旦これでいく。既にこれですら研修生組が破綻している。

In [5]:
def format_line(line):
   # 日本語以外の文字を排除(韓国語とか中国語とかヘブライ語とか)
    jp_chartype_tokenizer = nltk.RegexpTokenizer(u'([ぁ-んー]+|[ァ-ンー]+|[\u4e00-\u9FFF]+|[ぁ-んァ-ンー\u4e00-\u9FFF]+|[.!?])')
    line = "".join(jp_chartype_tokenizer.tokenize(line))
    line = re.sub('ー+', 'ー', line) # 連続長音
    return line

記号・数値・アルファベットは出てこない割にゴミになりやすいから全部消したほうが早い  
長音・びっくり・はてな三点リーダだけ残して後は目で見て入れる  
NewGame以外の漫画だとまた別かも

In [61]:
comic_num = 1
base_path = './output/response_newgame{}/'.format(comic_num)
output_file = codecs.open('./output/newgame_script{}.tsv'.format(comic_num), 'w', 'utf-8')

file_list = [r for r in os.listdir(base_path) if r.endswith('.json')]
for response_file in sorted(file_list, key=lambda x: int(x[8:-5])):
    page_num = re.search('\d+', response_file).group()
    response_path = os.path.join(base_path, response_file)
    with codecs.open(response_path, 'r', 'utf-8') as f:
        response = json.loads(f.read())
    if not 'responses' in response: 
        continue
    for koma_num, r in enumerate(response['responses']):
        if not 'textAnnotations' in r: 
            continue
        lines = r['textAnnotations'][0]['description'].split('\n')
        lines = map(format_line, lines)
        lines = filter(lambda l: len(l)>1, lines)
        lines = list(lines)
        if len(lines) == 0: 
            continue

        # 同コマで複数キャラが喋るのでlinesはバラして出力する
        for line in lines:
            output = u'\t'.join(map(str, [page_num, koma_num+1,  line.replace('"', '')])) + u'\n'    
            output_file.write(output)
output_file.close()

## 同じ領域を二回書き出している部分があって、片方が結構良い。悪い方は縦書きになっている。
## か？ が何故かかぶりやすい

# コマにラベルをつける

**シリーズ名/character/巻数.csv**   
**ページ数, コマ数,ラベル欄**  
同コマに複数キャラが居る可能性があるため、ページ・コマが同じレコードが複数できる